In [ ]:
import pandas as pd
import numpy as np
import json
import plotly
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
import os
from PIL import Image

In [ ]:
tidy_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tidy.csv")
tidy_df.columns

Index(['gameID_eventID', 'period', 'dateTime', 'gameID', 'team', 'goal', 'x',
       'y', 'shooter', 'goalie', 'shotType', 'emptyNet', 'strength',
       'gameType', 'home', 'away', 'season'],
      dtype='object')

In [ ]:
tidy_df.head()

,gameID_eventID,period,dateTime,gameID,team,goal,x,y,shooter,goalie,shotType,emptyNet,strength,gameType,home,away,season
0,2017020495_54,1,2017-12-16T03:08:32Z,2017020495,Vancouver Canucks,False,-78.0,4.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
1,2017020495_55,1,2017-12-16T03:08:41Z,2017020495,Vancouver Canucks,False,-81.0,0.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
2,2017020495_57,1,2017-12-16T03:09:40Z,2017020495,Vancouver Canucks,True,-79.0,-1.0,Markus Granlund,Martin Jones,Deflected,False,Power Play,R,Vancouver Canucks,San Jose Sharks,20172018
3,2017020495_59,1,2017-12-16T03:10:50Z,2017020495,Vancouver Canucks,False,-67.0,20.0,Brock Boeser,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
4,2017020495_61,1,2017-12-16T03:12:03Z,2017020495,Vancouver Canucks,False,-38.0,-1.0,Alex Biega,Martin Jones,Slap Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018


In [ ]:
# new_df = tidy_df[tidy_df['season']==20162017]
new_df = tidy_df
len(new_df.gameID.unique())

6194

In [ ]:
new_df = new_df.rename(columns={'gameID':'gamePk', 'team':'shooterTeam'})

In [ ]:
df = new_df
df.head()

,gameID_eventID,period,dateTime,gamePk,shooterTeam,goal,x,y,shooter,goalie,shotType,emptyNet,strength,gameType,home,away,season
0,2017020495_54,1,2017-12-16T03:08:32Z,2017020495,Vancouver Canucks,False,-78.0,4.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
1,2017020495_55,1,2017-12-16T03:08:41Z,2017020495,Vancouver Canucks,False,-81.0,0.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
2,2017020495_57,1,2017-12-16T03:09:40Z,2017020495,Vancouver Canucks,True,-79.0,-1.0,Markus Granlund,Martin Jones,Deflected,False,Power Play,R,Vancouver Canucks,San Jose Sharks,20172018
3,2017020495_59,1,2017-12-16T03:10:50Z,2017020495,Vancouver Canucks,False,-67.0,20.0,Brock Boeser,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
4,2017020495_61,1,2017-12-16T03:12:03Z,2017020495,Vancouver Canucks,False,-38.0,-1.0,Alex Biega,Martin Jones,Slap Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/20162017_clean.csv")
df['gameDurationSeconds'] = (pd.to_datetime(df['gameEnd_time'])-pd.to_datetime(df['gameStart_time'])).dt.total_seconds()
df.columns

Index(['gamePk', 'season', 'gameType', 'gameStart_time', 'gameEnd_time',
       'gameDuring', 'home', 'away', 'eventIdx', 'eventId', 'eventTypeId',
       'shootType', 'shooter', 'shooterTeam', 'goalie', 'eventPeriod',
       'eventPeriodType', 'x', 'y', 'gameDurationSeconds'],
      dtype='object')

In [ ]:
df.isna().sum()

gameID_eventID         0
period                 0
dateTime               0
gamePk                 0
shooterTeam            0
goal                   0
x                     16
y                     15
shooter                0
goalie              1801
shotType              49
emptyNet          352021
strength          351018
gameType               0
home                   0
away                   0
season                 0
dtype: int64

In [ ]:
df = df.dropna(subset=['shooterTeam', 'x','y'])
print(len(df.gamePk.unique()))
df.head()

6194


,gameID_eventID,period,dateTime,gamePk,shooterTeam,goal,x,y,shooter,goalie,shotType,emptyNet,strength,gameType,home,away,season
0,2017020495_54,1,2017-12-16T03:08:32Z,2017020495,Vancouver Canucks,False,-78.0,4.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
1,2017020495_55,1,2017-12-16T03:08:41Z,2017020495,Vancouver Canucks,False,-81.0,0.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
2,2017020495_57,1,2017-12-16T03:09:40Z,2017020495,Vancouver Canucks,True,-79.0,-1.0,Markus Granlund,Martin Jones,Deflected,False,Power Play,R,Vancouver Canucks,San Jose Sharks,20172018
3,2017020495_59,1,2017-12-16T03:10:50Z,2017020495,Vancouver Canucks,False,-67.0,20.0,Brock Boeser,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018
4,2017020495_61,1,2017-12-16T03:12:03Z,2017020495,Vancouver Canucks,False,-38.0,-1.0,Alex Biega,Martin Jones,Slap Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018


In [ ]:
df.isna().sum()

gameID_eventID         0
period                 0
dateTime               0
gamePk                 0
shooterTeam            0
goal                   0
x                      0
y                      0
shooter                0
goalie              1800
shotType              44
emptyNet          352012
strength          351012
gameType               0
home                   0
away                   0
season                 0
dtype: int64

In [ ]:
df['absX'] =  df['x'].apply(abs).apply(round)
df['absY'] = np.where(df["x"] < 0,-df["y"] + 42,df["y"] + 42)
df['absY'] =  df['absY'].apply(round)
df.head(3)

,gameID_eventID,period,dateTime,gamePk,shooterTeam,goal,x,y,shooter,goalie,shotType,emptyNet,strength,gameType,home,away,season,absX,absY
0,2017020495_54,1,2017-12-16T03:08:32Z,2017020495,Vancouver Canucks,False,-78.0,4.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018,78,38
1,2017020495_55,1,2017-12-16T03:08:41Z,2017020495,Vancouver Canucks,False,-81.0,0.0,Sam Gagner,Martin Jones,Wrist Shot,NaN,NaN,R,Vancouver Canucks,San Jose Sharks,20172018,81,42
2,2017020495_57,1,2017-12-16T03:09:40Z,2017020495,Vancouver Canucks,True,-79.0,-1.0,Markus Granlund,Martin Jones,Deflected,False,Power Play,R,Vancouver Canucks,San Jose Sharks,20172018,79,43


In [ ]:
def calc_avg_shot_freq(df):
    avg_freq = np.zeros([100,85])
    avg_freq_map = dict(df[['absX','absY']].apply(tuple, axis=1).value_counts())
    for i,j in avg_freq_map.keys():
        avg_freq[i,j] = avg_freq_map[(i,j)]

    # return avg_freq/(df.drop_duplicates(subset='gamePk')['gameDurationSeconds'].sum()/3600)
    return avg_freq/len(df.drop_duplicates(subset='gamePk'))

def calc_team_avg_shot_freq(df, team):
    avg_freq = np.zeros([100,85])
    team_df = df[df['shooterTeam']==team].copy()
    return calc_avg_shot_freq(team_df)

def clac_team_normalized_clean_shot_freq(df, team, sigma=5, thresh = 0.001):
    league_avg_shot_freq = calc_avg_shot_freq(df)/2 # devide by the factor of two as two teams play the game

    team_avg_shot_freq = calc_team_avg_shot_freq(df, team)
    avg_diff_shot_freq = team_avg_shot_freq - league_avg_shot_freq

    # smoothening and cleaning for the better visualization
    avg_diff_shot_freq = gaussian_filter(avg_diff_shot_freq, sigma=sigma)
    avg_diff_shot_freq[np.abs(avg_diff_shot_freq-0) <= thresh] = None
    return avg_diff_shot_freq

def get_shot_freq_plot_data(df):
    seasons = df['season'].unique()
    teams = df['shooterTeam'].unique()

    plot_data = {}
    for season in seasons:
        plot_data[season]={}
        season_df = df[df['season']==season]
        for team in teams:
            plot_data[season][team] = clac_team_normalized_clean_shot_freq(season_df, team, sigma=5, thresh = 0.001)
    return plot_data

plot_data = get_shot_freq_plot_data(df)

<ipython-input-34-7dece410acd6>:8: RuntimeWarning: invalid value encountered in divide
  return avg_freq/len(df.drop_duplicates(subset='gamePk'))


In [ ]:
df.shooterTeam.unique()

array(['Toronto Maple Leafs', 'Ottawa Senators', 'St. Louis Blues',
       'Chicago Blackhawks', 'Edmonton Oilers', 'Calgary Flames',
       'Los Angeles Kings', 'San Jose Sharks', 'Montréal Canadiens',
       'Buffalo Sabres', 'New York Rangers', 'New York Islanders',
       'Washington Capitals', 'Pittsburgh Penguins', 'Boston Bruins',
       'Columbus Blue Jackets', 'Detroit Red Wings',
       'Tampa Bay Lightning', 'New Jersey Devils', 'Florida Panthers',
       'Minnesota Wild', 'Carolina Hurricanes', 'Winnipeg Jets',
       'Anaheim Ducks', 'Dallas Stars', 'Nashville Predators',
       'Philadelphia Flyers', 'Colorado Avalanche', 'Arizona Coyotes',
       'Vancouver Canucks'], dtype=object)

In [ ]:
fig = go.Figure()
fig.add_layout_image(
        dict(
            source=Image.open("/content/nhl_rink.png"),
            xref="x",
            yref="y",
            x=-100,
            y=85,
            sizex=200,
            sizey=85,
            sizing="stretch",
            opacity=0.50,
            layer="above")
)

# fig.add_trace (
#             go.Contour(
#                 x=list(range(100)),
#                 y=list(range(85)),
#                 # z=plot_data[20172018]['Montréal Canadiens'].T,
#                 z=np.rot90(np.fliplr(plot_data[20172018]['Montréal Canadiens'])),
#                 colorscale='RdBu',
#                 reversescale=True,
#                 connectgaps= False,
#             )
#         )

season= 20172018
teams = list(plot_data[season].keys())
mask = [x for x in (np.eye(len(teams))==1)]
buttons = []
for i in range(len(teams)):
# for i in range(3):
    fig.add_trace (
            go.Contour(
                x=list(range(100)),
                y=list(range(85)),
                # z=plot_data[20172018]['Montréal Canadiens'].T,
                z=np.rot90(np.fliplr(plot_data[20172018][teams[i]])),
                colorscale='RdBu',
                reversescale=True,
                connectgaps= False,
            )
        )
    buttons.append(dict(method='update',
                       label=teams[i],
                       args=[{'visible':mask[i],
                              'title':teams[i],
                              'showlegend':True}]
                       ))

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=buttons
        )
    ])
fig.show()
fig.update_layout(
        title=dict(
            text= "<i> Shot Frequency of Team Compared to League Average for the Season 2017-2018 </i>",
            font={'size':18},
            y=0.075,
            x=0.5,
            xanchor= 'center',
            yanchor= 'top'))
fig.write_html(os.path.join('Q6_plot.html'),default_width='90%', default_height="100%", include_plotlyjs="cdn")